In [8]:
from surprise import Reader, Dataset, KNNWithMeans, accuracy
from surprise.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [4]:
from sklearn.model_selection import train_test_split

# 데이터 로드 및 전처리
df = pd.read_csv('../df.csv')
df.reset_index(inplace=True, drop=True)
df = df[['user_id', 'movie_id', 'rating']]
df = df[:10000]
train, test = train_test_split(df, test_size=0.2, random_state=42)

# 평점 범위를 1~10으로 설정
# 학습 및 테스트 데이터셋 분리
reader = Reader(rating_scale=(1, 10))
traindata = Dataset.load_from_df(train, reader)
testdata = Dataset.load_from_df(test, reader)
trainset = traindata.build_full_trainset()
testset = testdata.build_full_trainset().build_testset()

In [37]:
# 특정 사용자에 대한 영화 추천 함수
def get_recommendations(algo, user_id, df, n=20):
    # 사용자가 평가하지 않은 영화 목록 생성
    user_rated_movies = df[df['user_id'] == user_id]['movie_id'].tolist()
    all_movies = df['movie_id'].unique()
    unrated_movies = [movie for movie in all_movies if movie not in user_rated_movies]
    
    # 예상 평점 계산
    predictions = [algo.predict(user_id, movie) for movie in unrated_movies]
    predictions.sort(key=lambda x: x.est, reverse=True) #예상 평점 기준 정렬
    
    # 상위 n개의 영화 추천
    top_n_recommendations = predictions[:n]
    return [(pred.iid, pred.est) for pred in top_n_recommendations] # (영화id, 예상 평점)

KNNWithMeans

###### knnwithmeans - gridsearch

In [35]:
from surprise import Dataset, Reader, KNNWithMeans

# 하이퍼파라미터 그리드 설정
param_grid = {
    'k': [5, 10, 20, 30, 40, 50],  # 이웃 수
    'sim_options': {
        'name': ['pearson_baseline'],
        'user_based': [False]
    }
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5, n_jobs=-1) # 5-폴드 교차 검증
gs.fit(traindata)

# 최적의 하이퍼파라미터와 해당 성능 출력
print("Best RMSE score: ", gs.best_score['rmse'])
print("Best parameters: ", gs.best_params['rmse'])

Best RMSE score:  2.5307982523722172
Best parameters:  {'k': 20, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}}


In [36]:
# KNN 모델 학습
algo = KNNWithMeans(k=20, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

# 테스트 데이터셋으로 예측 및 RMSE 계산
test_pred = algo.test(testset)
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Item-based Model : Test Set
RMSE: 2.5873


2.5872518512214233

In [38]:
# 사용자 ID에 대한 추천 영화 목록 출력
user_id = 2  
recommended_movies = get_recommendations(algo, user_id, df, n=20)
print("추천된 영화 목록:")
for movie_id, est_rating in recommended_movies:
    print(f"영화 ID: {movie_id}, 예상 평점: {est_rating}")

추천된 영화 목록:
영화 ID: 10016, 예상 평점: 10
영화 ID: 10038, 예상 평점: 10
영화 ID: 100398, 예상 평점: 10
영화 ID: 100619, 예상 평점: 10
영화 ID: 100825, 예상 평점: 10
영화 ID: 10072, 예상 평점: 9.99134199134199
영화 ID: 100639, 예상 평점: 9.926941399458258
영화 ID: 10021, 예상 평점: 9.91324200913242
영화 ID: 10004, 예상 평점: 9.895684803001878
영화 ID: 10015, 예상 평점: 9.884915084915086
영화 ID: 100399, 예상 평점: 9.843373493975903
영화 ID: 10047, 예상 평점: 9.841666666666667
영화 ID: 10006, 예상 평점: 9.837398373983739
영화 ID: 10040, 예상 평점: 9.78714859437751
영화 ID: 10039, 예상 평점: 9.771929824561402
영화 ID: 10019, 예상 평점: 9.715447154471544
영화 ID: 10050, 예상 평점: 9.666666666666666
영화 ID: 10002, 예상 평점: 9.65
영화 ID: 10058, 예상 평점: 9.6375
영화 ID: 100356, 예상 평점: 9.594871794871796


KNNBaseline

###### knnbaseline - gridsearch

In [26]:
from surprise import Dataset, Reader, KNNBaseline

# 하이퍼파라미터 그리드 설정
param_grid = {
    'k': [5, 10, 20, 30, 40, 50],  # 이웃 수
    'sim_options': {
        'name': ['pearson_baseline'],
        'user_based': [False]
    },
    'bsl_options': {
        'method': ['als', 'sgd'],
        'n_epochs': [60,70,80,90,100],
        'reg_u': [10, 15],
        'reg_i': [5, 10]
    }
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=5, n_jobs=-1) # 5-폴드 교차 검증
gs.fit(traindata)

# 최적의 하이퍼파라미터와 해당 성능 출력
print("Best RMSE score: ", gs.best_score['rmse'])
print("Best parameters: ", gs.best_params['rmse'])

Best RMSE score:  2.276919103521916
Best parameters:  {'k': 10, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}, 'bsl_options': {'method': 'sgd', 'n_epochs': 60, 'reg_u': 10, 'reg_i': 5}}


In [29]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
bsl_options = {'method': 'sgd', 'n_epochs': 60, 'reg_u': 10, 'reg_i': 5}
algo = KNNBaseline(k=10, sim_options=sim_options, bsl_options=bsl_options)
algo.fit(trainset)

# 테스트 데이터셋으로 예측 및 RMSE 계산
test_pred = algo.test(testset)
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Item-based Model : Test Set
RMSE: 2.3003


2.3003212711139778

In [33]:
# 사용자 ID에 대한 추천 영화 목록 출력
user_id = 2  
recommended_movies = get_recommendations(algo, user_id, df, n=20)
print("추천된 영화 목록:")
for movie_id, est_rating in recommended_movies:
    print(f"영화 ID: {movie_id}, 예상 평점: {est_rating}")

추천된 영화 목록:
영화 ID: 10002, 예상 평점: 10
영화 ID: 10004, 예상 평점: 10
영화 ID: 10012, 예상 평점: 10
영화 ID: 10015, 예상 평점: 10
영화 ID: 10016, 예상 평점: 10
영화 ID: 10021, 예상 평점: 10
영화 ID: 100356, 예상 평점: 10
영화 ID: 10038, 예상 평점: 10
영화 ID: 100398, 예상 평점: 10
영화 ID: 100399, 예상 평점: 10
영화 ID: 10047, 예상 평점: 10
영화 ID: 10048, 예상 평점: 10
영화 ID: 10058, 예상 평점: 10
영화 ID: 100619, 예상 평점: 10
영화 ID: 100639, 예상 평점: 10
영화 ID: 10072, 예상 평점: 10
영화 ID: 100825, 예상 평점: 10
영화 ID: 10040, 예상 평점: 9.988759364002147
영화 ID: 10019, 예상 평점: 9.987716253905997
영화 ID: 10028, 예상 평점: 9.987189013757408


###### 아이템 유사도 + 협업 필터링

In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # 영화 데이터 로드
# movies_df = pd.read_csv('movies.csv')

# # 아이템 간의 유사도 계산을 위해 TF-IDF 벡터화
# tfidf = TfidfVectorizer(stop_words='english')
# tfidf_matrix = tfidf.fit_transform(movies_df['genres'].fillna(''))

# # 코사인 유사도 계산
# item_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# # 아이템 간의 유사도 행렬 출력
# print(item_similarity)

In [ ]:
# import numpy as np

# def item_based_collaborative_filtering(user_item_matrix, item_similarity_matrix, n_recommendations=5):
#     # 유사도 행렬과 사용자 선호도 행렬을 곱하여 예측 평점 행렬 계산
#     pred_ratings = np.dot(user_item_matrix, item_similarity_matrix) / np.sum(item_similarity_matrix, axis=1)
    
#     # 사용자가 이미 평가한 아이템은 제외
#     pred_ratings[user_item_matrix.nonzero()] = 0
    
#     # 각 사용자에 대해 상위 N개의 아이템 추천
#     top_n_recommendations = np.argsort(-pred_ratings, axis=1)[:, :n_recommendations]
    
#     return top_n_recommendations

# # 예시: 아이템 기반 협업 필터링 수행
# user_item_matrix = ...  # 사용자-아이템 행렬
# item_similarity_matrix = ...  # 아이템-아이템 유사도 행렬

# recommendations = item_based_collaborative_filtering(user_item_matrix, item_similarity_matrix)
# print(recommendations)